In [ ]:
import csv

Le fichier principal du module n°5, _dunkerque_smugglers_clean_20220126 - dunkerque_smugglers_clean_20220126.csv_ ci-dessous, vient du drive, [à cette adresse](https://docs.google.com/spreadsheets/d/1oMqQpDWt77SVUqrkogA0GoMU4MzHla2GAuzp4bClii4/edit#gid=1993901892)

In [ ]:
smugglers = list(csv.DictReader(open("data/dunkerque_smugglers_clean_20220126 - dunkerque_smugglers_clean_20220126.csv", 'r')))

Commande XSV pour isoler les flows dans toflit de produits smogglés par les bateaux smogglers identifiés dans Navigo

In [ ]:
# exports de thé vers l'Angleterre dans les années 80
!xsv search Angleterre -s partner ../../data/toflit18_all_flows.csv\
| xsv search "178.*" -s year\
| xsv search "Thé" -s product_revolutionempire\
| xsv search "Exports" -s export_import\
| xsv select line_number,year,customs_region,partner,product_orthographic,product_simplification,product_revolutionempire,value,quantity,quantity_unit,quantity_unit_orthographic,quantity_unit_metric,unit_price_metric,value_per_unit,product_revolutionempire\
> xsv_output_toflit18/the_price.csv

In [ ]:
# exports de feuilles de tabac vers l'Angleterre dans les années 80
!xsv search Angleterre -s partner ../../data/toflit18_all_flows.csv\
| xsv search "178.*" -s year \
| xsv search "feuilles de tabac" -s product_simplification\
| xsv search "Exports" -s export_import\
| xsv select line_number,year,customs_region,partner,product_orthographic,product_simplification,product_revolutionempire,value,quantity,quantity_unit,quantity_unit_orthographic,quantity_unit_metric,unit_price_metric,value_per_unit,product_revolutionempire\
> xsv_output_toflit18/tabac_price.csv

In [ ]:
# exports d'eaux-de-vie (catégorie large Révolution et Empire) vers l'Angleterre dans les années 80
!xsv search Angleterre -s partner ../../data/toflit18_all_flows.csv\
| xsv search "178.*" -s year\
| xsv search "Eaux-de-vie et liqueurs" -s product_revolutionempire\
| xsv search "Exports" -s export_import\
| xsv select line_number,year,customs_region,partner,product_orthographic,product_simplification,product_revolutionempire,value,quantity,quantity_unit,quantity_unit_orthographic,quantity_unit_metric,unit_price_metric,value_per_unit,product_revolutionempire\
> xsv_output_toflit18/eauxdevie_price.csv

In [ ]:
# PAS UTILISÉE
# exports de soie vers l'Angleterre dans les années 80
!xsv search Angleterre -s partner ../../data/toflit18_all_flows.csv\
| xsv search "178.*" -s year\
| xsv search "Soieries diverses" -s product_revolutionempire\
| xsv search "Exports" -s export_import\
| xsv select line_number,year,customs_region,partner,product_orthographic,product_simplification,product_revolutionempire,value,quantity,quantity_unit,quantity_unit_orthographic,quantity_unit_metric,unit_price_metric,value_per_unit,product_revolutionempire\
> xsv_output_toflit18/soieries_price.csv

Calculs des prix moyens/médians des produits ajoutés manuellement dans les fichiers _price.csv_, sauvegardés ensuite en _price_withaverage.csv_

### Prix moyens arbitrairement décidés par les experts que nous sommes

Alcool et eaux de vie : 0.62
Thés : 0.51
Feuilles de tabac : 0.36

### Quantification du smogglage

Application des prix moyens/médians calculés depuis les données toflit18 (fichiers xsv_output_toflit18/*_price_withaverage.csv) sur les données du smogglage extraites de navigo (fichier dunkerque_smugglers_clean_20220126 - dunkerque_smugglers_clean_20220126.csv)

In [ ]:
tea_smuggle_quantity = 0
tobacco_smuggle_quantity = 0
alcool_smuggle_quantity = 0

tea_smuggle_price = 0
tobacco_smuggle_price = 0
alcool_smuggle_price = 0
        
for line in smugglers:

    tobacco = line["tabac en feuilles (livres poids)"]
    if tobacco:
        tobacco_quantity = int(tobacco.replace("\xa0", ""))
        tobacco_price = tobacco_quantity * 0.36
        line["tobacco_price"] = tobacco_price

        tobacco_smuggle_quantity += tobacco_quantity
        tobacco_smuggle_price += tobacco_price

    tea = line["thé (livres poids)"]
    if tea:
        tea_quantity = int(tea.replace("\xa0", ""))
        tea_price = tea_quantity * 0.51
        line["tea_price"] = tea_price

        tea_smuggle_quantity += tea_quantity
        tea_smuggle_price += tea_price

    genieve = line["geniève (pintes de Paris)"]
    eauxdevie = line["eau-de-vie (pintes de Paris)"]
    tafia = line["Taffia (pintes de Paris)"]
    liqueurs = line["liqueurs (pintes)"]
    if genieve:
        genieve_quantity = int(genieve.replace("\xa0", "")) * 0.952
        genieve_price = genieve_quantity * 0.62
        alcool_smuggle_price += genieve_price
        alcool_smuggle_quantity += genieve_quantity
    if eauxdevie:
        eauxdevie_quantity = int(eauxdevie.replace("\xa0", "")) * 0.952
        eauxdevie_price = eauxdevie_quantity * 0.62
        alcool_smuggle_price += eauxdevie_price
        alcool_smuggle_quantity += eauxdevie_quantity
    if tafia:
        tafia_quantity = int(tafia.replace("\xa0", "")) * 0.952
        tafia_price = tafia_quantity * 0.62
        alcool_smuggle_price += tafia_price
        alcool_smuggle_quantity += tafia_quantity
    if liqueurs:
        liqueurs_quantity = int(liqueurs.replace("\xa0", "")) * 0.952
        liqueurs_price = liqueurs_quantity * 0.62
        alcool_smuggle_price += liqueurs_price
        alcool_smuggle_quantity += liqueurs_quantity

In [ ]:
print("Export smugglés de tabac en livres tournois : ", round(tobacco_smuggle_price))
print("Export smugglés de tabac en livres poids : ", tobacco_smuggle_quantity)
print("Export smugglés de thés en livres tournois : ", round(tea_smuggle_price))
print("Export smugglés de thés en livres poids : ", tea_smuggle_quantity)
print("Export smugglés des eaux-de-vie et liqueurs en livres tournois : ", round(alcool_smuggle_price))
print("Export smugglés des eaux-de-vie et liqueurs en litres : ", round(alcool_smuggle_quantity))

In [ ]:
print("Export smogglés totaux des 3 produits : ", round(tea_smuggle_price + tobacco_smuggle_price + alcool_smuggle_price), " livres tournois")

Recherche des flux toflit18 "vins" afin de construire un prix "moyen" pour les eaux-de-vie, liqueurs ET vins, puis joindre nos analyses avec le module 4

In [ ]:
!xsv search Angleterre -s partner ../../data/toflit18_all_flows.csv\
| xsv search "178.*" -s year\
| xsv search "vins? " -s product_revolutionempire\
| xsv search "Exports" -s export_import\
| xsv select line_number,year,customs_region,partner,product_orthographic,product_simplification,product_revolutionempire,value,quantity,quantity_unit,quantity_unit_orthographic,quantity_unit_metric,unit_price_metric,value_per_unit,product_revolutionempire\
> xsv_output_toflit18/vins_price.csv

In [ ]:
vins_smuggle_quantity = 0
vins_smuggle_price = 0
    
for line in smugglers:         
    vin_rouge = line["vin rouge (pintes)"]
    vin_fin = line["vin fin (pintes)"]
    barriques = line["vin (barriques)"]
    pots = line["vin (pots)"]
    
    if vin_rouge:
        vin_rouge_quantity = int(vin_rouge.replace("\xa0", ""))
        vin_rouge_price = vin_rouge_quantity * 1.8
        vins_smuggle_price += vin_rouge_price
        vins_smuggle_quantity += vin_rouge_quantity
        
    if vin_fin:
        vin_fin_quantity = int(vin_fin.replace("\xa0", ""))
        vin_fin_price = vin_fin_quantity * 1.8
        vins_smuggle_price += vin_fin_price
        vins_smuggle_quantity += vin_fin_quantity

In [ ]:
print("Export smugglés de vins en livres tournois : ", round(vins_smuggle_price))
print("Export smugglés de vins en litres : ", vins_smuggle_quantity)

Abandon de la piste de jointure des analyses avec le module 4 : le module 4 n'a pas de données pour 1787

Recherche des flux toflit18 "sucre brut" afin d'en trouver le prix

In [ ]:
!xsv search Angleterre -s partner_grouping ../../data/toflit18_all_flows.csv\
| xsv search "178.*" -s year| xsv search "sucre brut" -s product_simplification\
| xsv search "Exports" -s export_import\
| xsv select line_number,year,customs_region,partner,product_orthographic,product_simplification,product_revolutionempire,value,quantity,quantity_unit,quantity_unit_orthographic,quantity_unit_metric,unit_price_metric,value_per_unit\
> xsv_output_toflit18/sucre_price.csv

In [ ]:
sucre_smuggle_quantity = 0
sucre_smuggle_price = 0
    
for line in smugglers:         
    sucre = line["sucre brut (livres poids)"]

    if sucre:
        sucre_quantity = int(sucre.replace("\xa0", ""))
        sucre_price = sucre_quantity * 0.35
        sucre_smuggle_price += sucre_price
        sucre_smuggle_quantity += sucre_quantity

In [ ]:
print("Export smugglés de sucre brut en livres tournois : ", round(sucre_smuggle_price))
print("Export smugglés de sucre brut en livre poids : ", sucre_smuggle_quantity)

And now for something completely different

In [ ]:
!xsv search Angleterre -s partner_grouping ../../data/toflit18_all_flows.csv\
| xsv search "178.*" -s year| xsv search "Café" -s product_revolutionempire\
| xsv search "Exports" -s export_import\
| xsv select line_number,year,customs_region,partner,product_orthographic,product_simplification,product_revolutionempire,value,quantity,quantity_unit,quantity_unit_orthographic,quantity_unit_metric,unit_price_metric,value_per_unit\
> xsv_output_toflit18/cafe_price.csv

In [ ]:
cafe_smuggle_quantity = 0
cafe_smuggle_price = 0
    
for line in smugglers:         
    cafe = line["café (livres poids)"]

    if cafe:
        cafe_quantity = int(cafe.replace("\xa0", ""))
        cafe_price = cafe_quantity * 0.95
        cafe_smuggle_price += cafe_price
        cafe_smuggle_quantity += cafe_quantity

In [ ]:
print("Export smugglés de café en livres tournois : ", round(cafe_smuggle_price))
print("Export smugglés de café en livre poids : ", cafe_smuggle_quantity)

Calcul du montant des produits smogglés dont la valeur est déjà présente dans les données des smugglers

La catégorie Varia contient les produits suivants :

mousseline des Indes (livres tournois)
mousselines et mouchoirs (livres tournois)
mouchoirs de soie (livres tournois)
bandanoes (valeur livres tournois)
soie et mousseline (livres tournois)
marchandises des Indes (livres tournois)
soieries (livres tournois)
Cambray (en livres tournois)
nacres de perle (livres tournois)
porcelaine (livres tournois)

In [ ]:
varia_smuggle_price = 0

for line in smugglers:
    for product in line.keys():
        if "livres tournois" in product:
            if line[product]:
                varia_smuggle_price += int(line[product].replace("\xa0", ""))

print("Export smugglés de catégorie varia en livres tournois : ", round(varia_smuggle_price))
